# Agentic AI : Pydantic Models

Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.

In [1]:
!pip install pydantic

### Making Models Using Pydantic BaseModel (Validation included)

In [2]:
from pydantic import BaseModel, ValidationError

In [3]:
class Person(BaseModel):
    name:str
    age:int
    city:str

person = Person(name="Biparnak", age=25, city="Hyderabad")
print(person)

name='Biparnak' age=25 city='Hyderabad'


If the we declear the city as an int it will give an error as pydantic does data validation

In [4]:
try:
    person = Person(name="Biparnak", age=25, city=1)
except ValidationError as e:
    print("Validation failed:")
    print(e.json(indent=2))

Validation failed:
[
  {
    "type": "string_type",
    "loc": [
      "city"
    ],
    "msg": "Input should be a valid string",
    "input": 1,
    "url": "https://errors.pydantic.dev/2.11/v/string_type"
  }
]


### Making Models Using data class (Validation excluded)

In [5]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

person1 = Person(name="Biparnak", age=25, city="Hyderabad")
print(person1)


Person(name='Biparnak', age=25, city='Hyderabad')


If we do the same with dataclass (non pydantic) no validation is there and the city is marked as an int which it should not

In [6]:
person1 = Person(name="Biparnak", age=25, city=1)
print(person1)

Person(name='Biparnak', age=25, city=1)


### Model with Optional Fields

Add optional fields using Python's Opitonal type



Definition:

- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided



In [7]:
from typing import Optional

class Employee(BaseModel):
    id: int
    name : str
    department : str
    salary:  Optional[float] = None #optional with default value as nonw
    is_active: Optional[bool] = True #Optional with default value as True

In [8]:
# Examples with and without optional fields

emp1 = Employee(id=1,name="John", department="SDE")
print(emp1)


emp2 = Employee(id=2,name="Jane", department="HR", salary=6000.0, is_active=False)
print(emp2)

#Auto Type Casting
emp3 = Employee(id=3,name="Joey", department="HR", salary=4000, is_active=False)
print(emp2)

id=1 name='John' department='SDE' salary=None is_active=True
id=2 name='Jane' department='HR' salary=6000.0 is_active=False
id=2 name='Jane' department='HR' salary=6000.0 is_active=False


### Making Models With List as Field

In [9]:
from pydantic import BaseModel
from typing import List

class Classroom(BaseModel):
    room_number : str
    students : List[str]  #list of strings
    capacity: int

In [10]:
#create class room
classroom = Classroom(
    room_number = "A101",
    students=["Biparnak","Bob","Charlie"],
    capacity=30
)
print(classroom)

room_number='A101' students=['Biparnak', 'Bob', 'Charlie'] capacity=30


In [11]:
#type Casting of tuple to List (Dicts won't work)
classroom_1 = Classroom(
    room_number = "A101",
    students=("Biparnak","Bob","Charlie"),
    capacity=30
)
print(classroom_1)

room_number='A101' students=['Biparnak', 'Bob', 'Charlie'] capacity=30


### Error Handling  Using Try Catch

In [12]:
try:
    invalid_val = Classroom(room_number="A1", students=["Biparnak",123], capacity=30)
except ValueError as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


### Model with Nested Models

In [13]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    zip_code: int

class Customer(BaseModel):
    customer_id: int
    name: str
    address: Address  # Nested model

# Create a customer with nested address
customer = Customer(
    customer_id=1,
    name="Emma",
    address={"street": "123 Main St", "city": "Boston", "zip_code": "02108"}
)
print(customer)

customer_id=1 name='Emma' address=Address(street='123 Main St', city='Boston', zip_code=2108)


### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.


In [14]:
from pydantic import BaseModel,Field
class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float= Field(gt=0,le=1000) #greater than 0, less than or equal to 1000
    quantity:int=Field(ge=0)

# Valid instance
item = Item(name="Book", price=10, quantity=10)

print(item)

name='Book' price=10.0 quantity=10


In [15]:
from pydantic import BaseModel, Field

class User(BaseModel):
    username: str = Field(..., description="Unique username for the user")
    age: int = Field(default=18, description="User age, defaults to 18")
    email: str = Field(default_factory=lambda: "user@example.com", description="Default email address")

# Examples
user1 = User(username="alice")
print(user1)  # username='alice' age=18 email='user@example.com'

user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)  # username='bob' age=25 email='bob@domain.com'

username='alice' age=18 email='user@example.com'
username='bob' age=25 email='bob@domain.com'


In [16]:
print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique username for the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
